<a href="https://colab.research.google.com/github/datarobot-community/custom-models/blob/master/drum_overview/Main_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using MLOps DRUM to test your custom models
**Author**: Tim Whittaker

#### Scope
We'll get our hands dirty by:

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Show a simple example app connected to the REST API
* H2O, Keras, XGBoost, and DataRobot
* Add a DataRobot remote agent if you are interested in further model monitoring



In [ ]:
#Clone the repository
!git clone https://github.com/datarobot-community/mlops-examples

In [ ]:
!pip install -r /content/mlops-examples/drum_overview/requirements.txt

## Train a regression model

A simple RandomForestRegressor to predict house prices using the Concrete compressive strength dataset found in this paper:

`I-Cheng Yeh, "Modeling of strength of high performance concrete using artificial neural networks," Cement and Concrete Research, Vol. 28, No. 12, pp. 1797-1808 (1998).`.

In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

# Read the train and test data
TRAIN_DATA_REG = "data/concrete_train.csv"  # 14 features
TEST_DATA_REG = "data/concrete_test.csv"  # 13 features - target is removed

reg_X_train = pd.read_csv(TRAIN_DATA_REG)
reg_Y_train = reg_X_train.pop('/content/mlops-examples/drum_overview/concrete_compressive_strength')

reg_X_test = pd.read_csv(TEST_DATA_REG)

# Fit the model
rf = RandomForestRegressor()
rf.fit(reg_X_train, reg_Y_train)

# Pickle the file and write it to the file system
with open("custom_model_reg/reg_rf_model.pkl", 'wb') as pkl:
    pickle.dump(rf, pkl)
    
# Call predict to confirm it works
rf.predict(reg_X_test)

array([47.80232256, 15.75251333, 30.46034641, 11.20301973,  4.45028489,
       25.49138251, 41.3377946 ,  5.41167644, 33.94650944, 35.69269015,
       51.69296668, 20.23232848,  9.14960852,  8.83707594, 39.50436169,
        9.89364276,  5.97656413, 10.5134541 , 43.69996794, 48.86625297,
       12.70730468, 16.61245538, 27.46318735, 35.84154112, 38.40258593,
       14.28827315, 17.13399571, 31.96783053, 38.01614842, 39.44389464,
       15.31419275, 12.83106562, 33.51324962, 40.06370599, 40.72231415,
       11.62349667, 17.63828535, 17.32331203, 24.56273418, 26.81768613,
       44.41539286, 53.00673266, 36.9936762 , 14.86864647, 22.59005372,
       16.97513945, 25.03654209, 32.70590375, 36.06022518, 44.72348127,
       50.90934962, 56.08790733, 65.72064548, 69.21720482, 70.89095705,
       72.33030715, 18.33247736, 23.07090117, 28.71617898, 30.12548171,
       31.80814789, 32.9065521 ,  9.59848498, 15.16150829, 22.34782011,
       26.24662763, 32.99375013, 38.99345308, 40.33700738, 15.03

## Testing the Model

Input the prediction dataset that includes all features except the target feature.

In [ ]:
%%sh 
drum perf-test --code-dir /content/mlops-examples/drum_overview/custom_model_reg --input /content/mlops-examples/MLOps_DRUM/data/concrete_test.csv  --target-type regression

## Validating the Model

In [6]:
%%sh 
drum validation --code-dir /content/mlops-examples/drum_overview/custom_model_reg \
--input /content/mlops-examples/drum_overview/data/concrete_test.csv \
--target-type regression > drum_validation.log

In [7]:
%%sh
tail drum_validation.log



Validation checks results
      Test case          Status   Details
Basic batch prediction   PASSED          
Null value imputation    PASSED          


# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to start making predictions with it.  To do this, we'll leverage DRUM and it's ability to natively handle our scikit learn model, all we need to do is tell DRUM where it resides as well as the data we wish to score.  

There are a lot of frameworks which DRUM supports nateively, but for those which DRUM doesn't support of these shelf, we'll just need to create some custom hooks so DRUM.  In this example, we'll highlight some very simple custom hooks, and will provide links to more complex examples.  

In [10]:
%%sh 
drum score --code-dir /content/mlops-examples/drum_overview/custom_model_reg \
--input /content/mlops-examples/drum_overview/data/concrete_test.csv \
--output /content/mlops-examples/drum_overview/data/predictions.csv --target-type regression

In [11]:
pd.read_csv("/content/mlops-examples/drum_overview/data/predictions.csv").head()

,Predictions
0,25.941
1,22.129
2,34.559
3,33.845
4,35.273


# Start the inference server locally

Batch scoring can be very useful, but the utility DRUM offers does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command - using the `server` mode instead of `score` model.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook

In [13]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [15]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/mlops-examples/drum_overview/custom_model_reg", 
              "--address", "0.0.0.0:6789", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
              "--verbose"
              ]

In [16]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [1]:
## confirm the server is running
time.sleep(10) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", "http://0.0.0.0:6789").content

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple python function to pass the data over appropriately.  We'll leverage the same function in our simple flask app a little later.  

In [ ]:
def score(data, port = "6789"):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [ ]:
# %%timeit
scoring_data = pd.read_csv("/content/mlops-examples/drum_overview/data/concrete_test.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

In [ ]:
requests.request("GET", "http://0.0.0.0:6789/").content

In [ ]:
inference_server.terminate()
inference_server.stdout.readlines()

In [ ]:
#Stop the flask server
%%sh
fuser -n tcp -k 6789

## Value Prop

One may ask, what is the benefit to be had here?  Well, first of, there is not need for me to write an api to get the model up and running.  Second, DRUM allows me to abstract the framework away (provided I'm using one that is natively supported, or I can write enough python so that DRUM understands how to hook up to the model.  

For example, I could hot swap models as I see fit

While we will run through several other frameworks with in `score` you can bet they are supported in `server` mode as well!